In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from src.template import template_ilo

/home/andrey/anaconda3/envs/aml_project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Model and tokenizer names{
model_name = "google/gemma-1.1-2b-it" #"mistralai/Mistral-7B-Instruct-v0.2" #"google/gemma-1.1-2b-it" #"google/gemma-7b-it"

# Load the model and tokenizer
device = torch.device("cuda:1")
model = AutoModelForCausalLM.from_pretrained(model_name).eval().to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.80it/s]


In [7]:
model_params = {
    "max_new_tokens": 700,
}
COURSE_NAME = "Big Data Technologies And Analytics"
COURSE_DESCRIPTION = "This course covers the following concepts: Advanced distributed data organization; Advanced distributed data processing."
content = f"{template_ilo}\nCOURSE: {COURSE_NAME}\nDESCRIPTION: {COURSE_DESCRIPTION}\nINTENDED_LEARNING_OUTCOMES:" #COURSE_TOPICS:"
message = [ 
    {
        "role": "user",
        "content": content,
    },
]
prompt = tokenizer.apply_chat_template(
        message, tokenize=False, add_generation_prompt=True
    )
input_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
generated_ids = model.generate(
    input_ids=input_ids.to(device),
    **model_params,
)
response = tokenizer.decode(generated_ids[0])

In [8]:
print(response[len(prompt):])

```json
{
  "COURSE": "Big Data Technologies and Analytics",
  "DESCRIPTION": "This course provides an introduction to the rapidly evolving field of big data. Students will learn about the challenges and opportunities of big data, as well as the latest techniques and tools for processing, analyzing, and visualizing large datasets. Topics covered include distributed data storage, data processing frameworks, machine learning algorithms for big data, and cloud-based big data solutions.",
  "INTENDED_LEARNING_OUTCOMES": {
    "What is the main purpose of this course?": "The main purpose of this course is to equip students with the knowledge and skills necessary to leverage big data technologies and tools in various industries and domains.",
    "ILOs defined at three levels": {
      "Level 1: What concepts should a student know/remember/explain?": [
        "Fundamentals of distributed data storage systems",
        "Distributed data processing frameworks",
        "Machine learning algor

In [9]:
course_topics = response[len(prompt):].split('"INTENDED_LEARNING_OUTCOMES": ')[1]
start_index = course_topics.index("{")
end_index = course_topics.rindex("}")# + 1
print(course_topics[start_index: end_index])

{
    "What is the main purpose of this course?": "The main purpose of this course is to equip students with the knowledge and skills necessary to leverage big data technologies and tools in various industries and domains.",
    "ILOs defined at three levels": {
      "Level 1: What concepts should a student know/remember/explain?": [
        "Fundamentals of distributed data storage systems",
        "Distributed data processing frameworks",
        "Machine learning algorithms for big data",
        "Cloud-based big data solutions"
      ],
      "Level 2: What basic practical skills should a student be able to perform?": [
        "Implement distributed data processing algorithms",
        "Analyze and visualize big data using machine learning techniques",
        "Deploy big data solutions on cloud platforms",
        "Evaluate the performance of big data processing systems"
      ],
      "Level 3: What complex comprehensive skills should a student be able to apply in real-life sc

In [10]:
import ast
res = {"Intended Learning Outcomes": ast.literal_eval(course_topics[start_index: end_index])}
import json
with open('bd_gemma_ilo.json', 'w', encoding='utf-8') as f:
    json.dump(res, f, ensure_ascii=False, indent=4)